In [1]:
import os
import sys
sys.path.append(f'{os.getcwd()}/../../')
sys.path.append(f'{os.getcwd()}/analytic')

In [2]:
import numpy             as np
import scipy             as sp
import magritte.setup    as setup
import magritte.core     as magritte
import magritte.mesher   as mesher
import healpy

dimension = 3
nrays     = 12*3**2
nspecs    = 5
nlspecs   = 1
nquads    = 15

r_in   = 1.0E13   # [m]
r_out  = 7.8E16   # [m]
nH2_in = 2.0E13   # [m^-3]
X_mol  = 1.0E-8   # [.]
temp   = 20.0     # [K]
turb   = 150.00   # [.]


def nH2(r):
    return nH2_in * np.power(r_in/r, 2.0)

def nTT(r):
    return X_mol * nH2(r)


scale_max = 0.07 * r_out
scale_min = 0.1 * r_in
scale_cte = 0.1 * r_in
scale_fun = f"{scale_cte / r_in**2} * (x*x + y*y + z*z)"


meshName = 'test_model.vtk'

# mesher.create_mesh_from_function(
#     meshName       = meshName,
#     boundary       = mesher.boundary_sphere_in_sphere(
#                          radius_in  = r_in,
#                          radius_out = r_out),
#     scale_min      = scale_min,
#     scale_max      = scale_max,
#     scale_function = scale_fun )

In [3]:
mesh = mesher.Mesh(meshName)

npoints = len(mesh.points)
nbs     = [n for sublist in mesh.neighbors for n in sublist]
n_nbs   = [len(sublist) for sublist in mesh.neighbors]


modelFile = 'test_model.hdf5'
lamdaFile = '../data/test.txt'


model = magritte.Model ()
model.parameters.set_spherical_symmetry(False)
model.parameters.set_pop_prec          (1.0e-6)
model.parameters.set_model_name        (modelFile)
model.parameters.set_dimension         (dimension)
model.parameters.set_npoints           (npoints)
model.parameters.set_nrays             (nrays)
model.parameters.set_nspecs            (nspecs)
model.parameters.set_nlspecs           (nlspecs)
model.parameters.set_nquads            (nquads)

model.geometry.points.position.set(mesh.points)
model.geometry.points.velocity.set(np.zeros((npoints, 3)))

model.geometry.points.multiscale.set_all_neighbors(n_nbs, nbs)
rs = np.linalg.norm(mesh.points, axis=1)

model.chemistry.species.abundance = [[     0.0, nTT(r), nH2(r),  0.0,      1.0] for r in rs]
model.chemistry.species.symbol    =  ['dummy0', 'test',   'H2', 'e-', 'dummy1']

model.thermodynamics.temperature.gas  .set( temp                 * np.ones(npoints))
model.thermodynamics.turbulence.vturb2.set((turb/magritte.CC)**2 * np.ones(npoints))

model.parameters.set_nboundary(len(mesh.boundary))
model.geometry.boundary.boundary2point.set(mesh.boundary)

model = setup.set_boundary_condition_CMB  (model)
model = setup.set_uniform_rays            (model)
model = setup.set_linedata_from_LAMDA_file(model, lamdaFile)
model = setup.set_quadrature              (model)

model.write()
# model.read ()

# model.compute_spectral_discretisation ()
# model.compute_inverse_line_widths     ()
# model.compute_LTE_level_populations   ()

# model.compute_radation_field()

# model.compute_level_populations       (False, 50)

0
6


In [4]:
import magritte.core as magritte

In [5]:
model = magritte.Model('test_model.hdf5')

In [7]:
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

0

In [8]:
model.compute_radiation_field_2nd_order_Feautrier()

0

In [6]:
model.parameters.spherical_symmetry()

False